In [1]:
import numpy as np
from Bio.SubsMat.MatrixInfo import blosum62 as blos 
from Bio.SubsMat.MatrixInfo import pam30 as pam

gap = 10

In [2]:
def e(s1, s2, i, j, weight_table):
    if(weight_table == "blosum62"):
        return blos.get((s1[i], s2[j])) or blos.get((s2[j], s1[i])) or 0
    elif(weight_table == "pam30"):
        return pam.get((s1[i], s2[j])) or pam.get((s2[j], s1[i])) or 0
    else:
        return 0

In [3]:
def create_matrix(s1, s2, weight_table):
    m = len(s1) + 1
    n = len(s2) + 1
    a = np.zeros((n, m))
        
    for i in range(1, n):
        for j in range(1, m):
            a[i][j] = np.max([a[i-1][j-1] + e(s1, s2, j-1, i-1, weight_table),
                             a[i-1][j] - gap,
                             a[i][j-1] - gap,
                             0])
    return a, n, m

In [4]:
def get_align(str1, str2, weight_table):
    a, n, m = create_matrix(str1, str2, weight_table)
    
    i_max, j_max = n - 1, m - 1
    
    for i in range(0, n):
        for j in range(0, m):
            if(a[i][j] >= a[i_max][j_max]):
                i_max = i
                j_max = j
                
    i, j = i_max, j_max
    
    s1, s2 = "", ""
    
    while(a[i][j] != 0):
        if(a[i-1][j-1] + e(str1, str2, j-1, i-1, weight_table) >= a[i][j-1] - gap and
           a[i-1][j-1] + e(str1, str2, j-1, i-1, weight_table) >= a[i-1][j] - gap):
            s1 += str1[j-1]
            s2 += str2[i-1]
            j -= 1
            i -= 1
        elif(a[i-1][j] - gap >= a[i-1][j-1] + e(str1, str2, j-1, i-1, weight_table) and
             a[i-1][j] - gap >= a[i][j-1] - gap):
            s2 += str2[i-1]
            s1 += "-"
            i -= 1
        elif(a[i][j-1] - gap >= a[i-1][j-1] + e(str1, str2, j-1, i-1, weight_table) and
             a[i][j-1] - gap >= a[i-1][j] - gap):
            s2 += "-"
            s1 += str1[j-1]
            j -= 1
            
    s1 = s1[::-1]
    s2 = s2[::-1]
    
    start_l1, start_l2 = 0, 0
    end_l1, end_l2 = 0, 0
    gap1, gap2 = 0, 0
    
    for i in range(0, len(s1)):
        if (s1[i] == '-'):
            gap1 += 1
    
    for i in range(0, len(s2)):
        if (s2[i] == '-'):
            gap2 += 1
    
    start_l1 = j_max - len(s1) + gap1
    start_l2 = i_max - len(s2) + gap2
    end_l1 = len(str1) - j_max
    end_l2 = len(str2) - i_max
    
    k = start_l1
    while(k != 0):
        s1 = str1[k - 1] + s1
        k -= 1
        
    k = start_l2
    while(k != 0):
        s2 = str2[k - 1] + s2
        k -= 1
        
    k = 0
    while(k != end_l1):
        s1 += str1[j_max + k]
        k += 1
        
    k = 0
    while(k != end_l2):
        s2 += str2[i_max + k]
        k += 1
    
    k = max(0, start_l2 - start_l1)
    while(k != 0):
        s1 = " " + s1
        k -= 1
        
    k = max(0, start_l1 - start_l2)
    while(k != 0):
        s2 = " " + s2
        k -= 1
        
    k = max(0, end_l2 - end_l1)
    while(k != 0):
        s1 += " "
        k -= 1
        
    k = max(0, end_l1 - end_l2)
    while(k != 0):
        s2 += " "
        k -= 1
    
    s_help = ""
    
    for i in range(0, len(s1)):
        if(s1[i] == s2[i]):
            s_help += "|"
        else:
            s_help += "."
    
    list_help = list(s_help)
    for i in range(0, max(start_l1, start_l2)):
        list_help[i] = ' '
    
    for i in range(len(s1) - max(end_l1, end_l2), len(s1)):
        list_help[i] = ' '
    s_help = "".join(list_help)
            
    return s1, s2, s_help

### BLOSUM62 TESTING

In [5]:
str1="TACGGGCCCGCTAC"
str2="TAGCCCTATCGGTCA"
s1, s2, s_help = get_align(str1, str2, "blosum62")
print(s1)
print(s_help)
print(s2)

TACGGGCCCGCTAC    
     ||||..|.     
   TAGCCCTATCGGTCA


In [6]:
str1="CCAAGGCT"
str2="CCAGAGAC"
s1, s2, s_help = get_align(str1, str2, "blosum62")
print(s1)
print(s_help)
print(s2)

CCA-AGGCT
|||.||.| 
CCAGAGAC 


In [7]:
str1="GGAGTGAGGGGAGCAGTTGGCTGAAGATGGTCCCCGCCGAGGGACCGGTGGGCGACGGCGAGCTGTGGCAGACCT"
str2="CATGCGGAGTGAGGGGAGCAGTTGGGAACAGATGGTCCCCGCCGAGGGACCGGTGGGCGACGGCCAGCTGTGGCA"
s1, s2, s_help = get_align(str1, str2, "blosum62")
print(s1)
print(s_help)
print(s2)

     GGAGTGAGGGGAGCAGTTGGCTGAAGATGGTCCCCGCCGAGGGACCGGTGGGCGACGGCGAGCTGTGGCAGACCT
     ||||||||||||||||||||....|||||||||||||||||||||||||||||||||||.||||||||||     
CATGCGGAGTGAGGGGAGCAGTTGGGAACAGATGGTCCCCGCCGAGGGACCGGTGGGCGACGGCCAGCTGTGGCA     


### PAM30 TESTING

In [8]:
str1="TACGGGCCCGCTAC"
str2="TAGCCCTATCGGTCA"
s1, s2, s_help = get_align(str1, str2, "pam30")
print(s1)
print(s_help)
print(s2)

TACGGGCCCGCTAC    
     ||||         
   TAGCCCTATCGGTCA


In [9]:
str1="CCAAGGCT"
str2="CCAGAGAC"
s1, s2, s_help = get_align(str1, str2, "pam30")
print(s1)
print(s_help)
print(s2)

CCA-AGGCT
|||.||.| 
CCAGAGAC 


In [10]:
str1="GGAGTGAGGGGAGCAGTTGGCTGAAGATGGTCCCCGCCGAGGGACCGGTGGGCGACGGCGAGCTGTGGCAGACCT"
str2="CATGCGGAGTGAGGGGAGCAGTTGGGAACAGATGGTCCCCGCCGAGGGACCGGTGGGCGACGGCCAGCTGTGGCA"
s1, s2, s_help = get_align(str1, str2, "pam30")
print(s1)
print(s_help)
print(s2)

     GGAGTGAGGGGAGCAGTTGGCTGAAGATGGTCCCCGCCGAGGGACCGGTGGGCGACGGCGAGCTGTGGCAGACCT
     ||||||||||||||||||||....|||||||||||||||||||||||||||||||||||.||||||||||     
CATGCGGAGTGAGGGGAGCAGTTGGGAACAGATGGTCCCCGCCGAGGGACCGGTGGGCGACGGCCAGCTGTGGCA     
